In [11]:
import pandas as pd
import numpy as np

In [12]:
scout_car = pd.read_csv('Step 1 - Cleaned v1.csv')
scout_car

,URL,make_model,Short Description,Body Type,Price,VAT,KM,Registration (YYYYMM),Horsepower,Type,...,Lane departure warning system,Night view assist,Passenger-side airbag,Power steering,Rear airbag,Side airbag,Tire pressure monitoring system,Traction control,Traffic sign recognition,Xenon headlights
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",201601.0,66 kW,Used,...,0,0,1,1,0,1,1,1,0,1
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",201703.0,141 kW,Used,...,0,0,1,1,0,1,1,1,0,1
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",201602.0,85 kW,Used,...,0,0,1,1,0,1,1,1,0,0
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,NaN,"73,000 km",201608.0,66 kW,Used,...,0,0,1,1,0,1,1,0,0,0
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,NaN,"16,200 km",201605.0,66 kW,Used,...,0,0,1,1,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,https://www.autoscout24.com//offers/renault-es...,Renault Espace,Blue dCi 200CV EDC Executive 4Control,Van,39950,VAT deductible,- km,NaN,147 kW,New,...,1,0,1,1,1,1,1,1,1,0
15915,https://www.autoscout24.com//offers/renault-es...,Renault Espace,"TCe 225 EDC GPF LIM Deluxe Pano,RFK",Van,39885,VAT deductible,"9,900 km",201901.0,165 kW,Used,...,1,0,1,1,0,1,1,1,1,0
15916,https://www.autoscout24.com//offers/renault-es...,Renault Espace,Blue dCi 200 EDC Initiale Paris Leder LED Navi...,Van,39875,VAT deductible,15 km,201903.0,146 kW,Pre-registered,...,1,0,1,1,0,1,0,1,1,0
15917,https://www.autoscout24.com//offers/renault-es...,Renault Espace,"Blue dCi 200CV EDC Business , NUOVA DA IMMATRI...",Van,39700,VAT deductible,10 km,201906.0,147 kW,Pre-registered,...,0,0,1,1,0,1,1,0,1,0


In [13]:
scout_car.isnull().sum()[scout_car.isnull().sum() > 0]

Short Description                     46
Body Type                             60
VAT                                 4513
Registration (YYYYMM)               1597
Type                                   2
Previous Owners                     6640
Next Inspection (YYYYMM)           12384
Inspection new                     11987
Warranty (months)                  13123
Offer Number                        3175
First Registration                  1597
Body Color                           597
Paint Type                          5772
Body Color Original                 3774
# of Doors                           212
# of Seats                           977
Model Code                         10941
Displacement (cc)                    496
Cylinders                           5680
Weight (kg)                         6974
Drive chain                         6858
CO2 Emission (g CO2/km (comb))      2436
Emission Class                      3628
Emission Label                     11934
Gears           

# Columns not necessary or irrelevant in machine learning?

These columns are complicated strings.

'URL'

'Short description'

'Offer number'

'Description' - But maybe I could use sentiment analysis and assign a numeric score accordingly? Would have to translate though, how reliable would translations be? I'll drop it for now but may come back to reincorporate it later.

In [14]:
scout_car.drop('URL', axis=1, inplace=True)
scout_car.drop('Short Description', axis=1, inplace=True)
scout_car.drop('Offer Number', axis=1, inplace=True)
scout_car.drop('description', axis=1, inplace=True)

# Handling missing values

For non-numeric columns, find the most common value (before converting to dummies).

In [15]:
# Identify non-numeric columns
non_numeric_cols = scout_car.select_dtypes(exclude=[np.number]).columns

# Fill nulls in each non-numeric column using mode within each make_model group
for col in non_numeric_cols:
    scout_car[col] = scout_car.groupby('make_model')[col]\
                              .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x))


/var/folders/cf/4zrkk0lx1nbdx9tpklpk4jc00000gn/T/ipykernel_92643/2522577465.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x))
/var/folders/cf/4zrkk0lx1nbdx9tpklpk4jc00000gn/T/ipykernel_92643/2522577465.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x))
/var/folders/cf/4zrkk0lx1nbdx9tpklpk4jc00000gn/T/ipykernel_92643/2522577465.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill,

In [16]:
scout_car

,make_model,Body Type,Price,VAT,KM,Registration (YYYYMM),Horsepower,Type,Previous Owners,Next Inspection (YYYYMM),...,Lane departure warning system,Night view assist,Passenger-side airbag,Power steering,Rear airbag,Side airbag,Tire pressure monitoring system,Traction control,Traffic sign recognition,Xenon headlights
0,Audi A1,Sedans,15770,VAT deductible,"56,013 km",201601.0,66 kW,Used,\n2\n,202106.0,...,0,0,1,1,0,1,1,1,0,1
1,Audi A1,Sedans,14500,Price negotiable,"80,000 km",201703.0,141 kW,Used,\n1\n,NaN,...,0,0,1,1,0,1,1,1,0,1
2,Audi A1,Sedans,14640,VAT deductible,"83,450 km",201602.0,85 kW,Used,\n1\n,NaN,...,0,0,1,1,0,1,1,1,0,0
3,Audi A1,Sedans,14500,VAT deductible,"73,000 km",201608.0,66 kW,Used,\n1\n,NaN,...,0,0,1,1,0,1,1,0,0,0
4,Audi A1,Sedans,16790,VAT deductible,"16,200 km",201605.0,66 kW,Used,\n1\n,NaN,...,0,0,1,1,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,- km,NaN,147 kW,New,\n1\n,NaN,...,1,0,1,1,1,1,1,1,1,0
15915,Renault Espace,Van,39885,VAT deductible,"9,900 km",201901.0,165 kW,Used,"['\n1\n', '\n', '7.4 l/100 km (comb)', '\n', '...",202201.0,...,1,0,1,1,0,1,1,1,1,0
15916,Renault Espace,Van,39875,VAT deductible,15 km,201903.0,146 kW,Pre-registered,"['\n1\n', '\n139 g CO2/km (comb)\n']",NaN,...,1,0,1,1,0,1,0,1,1,0
15917,Renault Espace,Van,39700,VAT deductible,10 km,201906.0,147 kW,Pre-registered,\n1\n,NaN,...,0,0,1,1,0,1,1,0,1,0


For numeric columns, find the *median* value. Or maybe find the mean and force rounding in the case of integers.

In [17]:
# --- Fill numeric columns with mean per make_model ---
numeric_cols = scout_car.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    mean_filled = scout_car.groupby('make_model')[col]\
                           .transform(lambda x: x.fillna(x.mean()))
    
    # If integer dtype, round and cast back
    if pd.api.types.is_integer_dtype(scout_car[col]):
        scout_car[col] = mean_filled.round().astype('Int64')  # Keeps nullable int type
    else:
        scout_car[col] = mean_filled

In [19]:
scout_car.to_csv('Validation csv 4.csv', index=False)